In [3]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
"""
Shows how to send an image with the Converse API to Anthropic Claude 3 Sonnet (on demand).
"""

import logging
import boto3


from botocore.exceptions import ClientError


logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


def generate_conversation(bedrock_client,
                          model_id,
                          input_text,
                          input_image):
    """
    Sends a message to a model.
    Args:
        bedrock_client: The Boto3 Bedrock runtime client.
        model_id (str): The model ID to use.
        input text : The input message.
        input_image : The input image.

    Returns:
        response (JSON): The conversation that the model generated.

    """

    logger.info("Generating message with model %s", model_id)

    # Message to send.

    with open(input_image, "rb") as f:
        image = f.read()

    message = {
        "role": "user",
        "content": [
            {
                "text": input_text
            },
            {
                    "image": {
                        "format": 'png',
                        "source": {
                            "bytes": image
                        }
                    }
            }
        ]
    }

    messages = [message]

    # Send the message.
    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages
    )

    return response


def main():
    """
    Entrypoint for Anthropic Claude 3 Sonnet example.
    """

    logging.basicConfig(level=logging.INFO,
                        format="%(levelname)s: %(message)s")

    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
    input_text = "이미지를 분석해서 얼굴의 특징을 뽑아내고, 얼굴색 및 성별에 맞는 중세시대 기사의 영웅적인 자화상을 만드는 프롬프트를 생성해줘"
    input_image = "skin_gender_img/amazon_big_size_photo.jpeg"

    try:

        bedrock_client = boto3.client(service_name="bedrock-runtime")

        response = generate_conversation(
            bedrock_client, model_id, input_text, input_image)

        output_message = response['output']['message']

        print(f"Role: {output_message['role']}")

        for content in output_message['content']:
            print(f"Text: {content['text']}")

        token_usage = response['usage']
        print(f"Input tokens:  {token_usage['inputTokens']}")
        print(f"Output tokens:  {token_usage['outputTokens']}")
        print(f"Total tokens:  {token_usage['totalTokens']}")
        print(f"Stop reason: {response['stopReason']}")

    except ClientError as err:
        message = err.response['Error']['Message']
        logger.error("A client error occurred: %s", message)
        print(f"A client error occured: {message}")

    else:
        print(
            f"Finished generating text with model {model_id}.")


if __name__ == "__main__":
    main()


INFO:__main__:Generating message with model anthropic.claude-3-sonnet-20240229-v1:0


Role: assistant
Text: 이 이미지는 검정색 배경에 중년 아시아 남성의 상반신 사진입니다. 그의 얼굴에는 미소를 띠고 있으며 밝고 친근한 표정을 짓고 있습니다. 그의 머리카락은 검은색이며 짧게 스타일링되어 있습니다.

프롬프트: 중세 시대 영웅적인 기사의 자화상을 그리되, 모델의 아시아 남성 얼굴 특징과 피부색을 반영하여 그리시오. 갑옷을 입고 검을 들고 있는 모습으로 표현하되, 얼굴 표정은 친근하고 생기 있게 표현하시오. 배경은 중세 시대 성곽 풍경으로 하시오.
Input tokens:  679
Output tokens:  266
Total tokens:  945
Stop reason: end_turn
Finished generating text with model anthropic.claude-3-sonnet-20240229-v1:0.
